# Introduction

This project is about analyzing app data from Google Play and Apple App stores. We'll pretend we're data analysts working for a company building Android and iOS apps. Our company only build apps that are free, and revenue comes from in-app ads. 

The goal is to use Python and Jupyter Notebook to profile the most profitable apps on the Google Play and Apple App stores Going through the data will help our developers understand what types of apps users gravitate towards.

The data for [Google Play][1] and [Apple App Store][2] can be downloaded at Kaggle.

[1]:https://www.kaggle.com/lava18/google-play-store-apps
[2]:https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps

## Opening and Exploring the Data

First we open the files and give them corresponding variable names. The header column is separated from the data for quick access. Here we will use the `explore_data()` function for exploration. It prints the rows in the list so they're readable, and finds the number of rows and columns if `rows_and_columns` is `True`. It assumes the input dataset doesn't have a header row.

The first few rows of each data set are printed along with the number of rows and columns. We also try and identify some columns that could help with our analysis.

In [ ]:
from csv import reader

#Open .csv files
file1 = open("AppleStore.csv", encoding='utf8')
file2 = open("googleplaystore.csv", encoding='utf8')

apple_file = reader(file1)
apple_apps_data = list(apple_file) #lists of list

google_file = reader(file2)
google_apps_data = list(google_file)

In [ ]:
#Separating the header from the data set 
apple_head = apple_apps_data[0]
apple_data = apple_apps_data[1:]

google_head = google_apps_data[0]
google_data = google_apps_data[1:]

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n') # new empty line for separation
        
    if rows_and_columns:
        print("Number of rows:", len(dataset))
        print("Number of columns:", len(dataset[0]))

#First few rows
#print("Apple Rows")
#explore_data(apple_data, 1, 3, True)
#print('\n')
#print("Google Rows")
#explore_data(google_data, 1, 3, True)

In [ ]:
#Exploring Columns
print(apple_head, '\n')
print(google_head, '\n')

The columns we could use need to be related to price (we develop free apps only) and the user ratings for the app. They're detailed in two tables here:

| Google Column Name | Description |
|:-----------:|:------------:|
| 'Rating' | User rating of the app |
| 'Installs' | Number of downloads |
| 'Price' | Price of the App |
| 'Type' | Whether an app is paid or free |

| Apple Column Name | Description |
|:---------:|:---------:|
| 'user_rating' | Average user rating (for all version) |
| 'user_rating_ver' | Average user rating (for current version) |
| 'Price' | Price of the app |